#抓取股票資料

## 月營收資料

In [0]:
import requests
import pandas as pd
import numpy as np
import datetime 
from bs4 import BeautifulSoup
import lxml
from io import StringIO
import time
import threading

In [0]:
#抓取月營收資料
def Monthly_revenue_Crawler(year,month):
    url='http://mops.twse.com.tw/nas/t21/sii/t21sc03_'+year+'_'+month+'_0.html'
    req = requests.get(url)
    req.encoding = 'big5'
    # 檢查HTTP Status
    if req.status_code != requests.codes.ok:
      print("FetchError"+req.status_code)
      return r.status_code
    else:
      soup = BeautifulSoup(req.text,'html.parser') 
      df_arr=[]
      table=[]
      html_df=[]
      for i in soup.find_all("table"):
        if i.find("table") != None:
          for t in range(len(i.find_all("table"))):
            table.append(i.find_all("table")[t])
      
      for t in table:
        html_table= pd.read_html(str(t))[0]
        if len(html_table)!=1:
          #print(html_table)
          html_df.append(html_table)
      #print(html_df[0])
      html_df.remove(html_df[0])
      df=pd.concat(html_df,axis=0)
      df.columns = df.columns.droplevel(0)
      df=df.drop(index=df[df['公司代號']=='合計'].index)

    return df

In [0]:
#抓取前cnt月的月營收資料
def Fetch_Monthly_revenue(cnt):
  count = cnt
  year =  datetime.datetime.now().year-1911
  month =  datetime.datetime.now().month-1
  result = {}
  while len(result) < count:
      result['%d-%d'%(year, month)] = Monthly_revenue_Crawler(str(year), str(month))

      month -=1
      if month == 0:
          year -=1
          month = 12
          
      time.sleep(5)
  return result

In [285]:
result=Fetch_Monthly_revenue(10)
#print(result)
for key in result.keys():
  #print(result[key]['公司代號'])
  result[key].index = result[key]['公司代號']

obj={}
for index,df in result.items():
  obj[index]=df['當月營收']

df = pd.DataFrame.from_dict(obj,orient='index')
df = df.sort_index()
df

,1201,1203,1210,1213,1215,1217,1220,1225,1232,1235,...,1104,1108,1110,6531,6166,6230,6579,8114,6278,3593
107-10,1772387,515554,6123232,36069,1871127,244422,250718,932581,1463517,3281,...,457241,300242,177767,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107-11,1633866,506912,6735632,26745,1765886,269672,280415,1029697,1466021,2042,...,421815,269955,138622,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107-12,1536834,521085,6339590,29860,1791014,269710,270544,936366,1463270,2236,...,465496,300034,183634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107-7,1899582,476840,6059127,87390,1898449,478884,225840,958539,1432128,3400,...,350896,264328,157245,403904.0,891612.0,700464.0,504334.0,846150.0,2730351.0,75656.0
107-8,1927383,571897,6280981,85863,1918613,668101,273493,985283,1441081,27967,...,354949,255469,98994,389919.0,842099.0,693975.0,583701.0,791595.0,2917566.0,78915.0
107-9,1857221,520954,6405654,36194,1769540,417077,238953,937936,1300638,5225,...,415742,256381,127730,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108-1,1583493,544108,7176513,63426,1885469,379511,261737,964100,1620137,1752,...,479518,353189,186293,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108-2,1358770,443171,4971488,39925,1463626,303197,143511,633413,1101574,1114,...,270638,168065,82927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108-3,1598137,514903,6130106,29047,1686202,313572,208035,880257,1437562,1830,...,437140,313093,147819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108-4,1695994,469207,6424220,28570,1739496,287677,209667,889193,1328619,2555,...,418992,323834,118229,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##財報資料

In [0]:
# 抓取財報資料函數
def fetch_financial_statements(year, season):
   # 查詢網址
    url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    
    # 查詢參數
    my_params = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'isQuery': 'Y',
        'TYPEK':'sii',
        'year':str(year),
        'season':str(season)
    }
    req = requests.post(url,params = my_params)
    
    # 檢查HTTP Status
    if req.status_code != requests.codes.ok:
      print("FetchError"+req.status_code)
      return r.status_code
    else:
      soup = BeautifulSoup(req.text, "lxml") 
      table=soup.find("table", class_= "hasBorder")
      df = pd.read_html(str(table))[0]
      #print(df[1])
      #print(df.columns)
      #print(df.loc[0])
      df.columns=df.loc[0]
      df=df.drop([0])
      #print(df)
      df=df.drop(index=df[df['公司代號']=='公司代號'].index)
      #print(df)
      return df

In [254]:
# 抓取至今前三年各季財報資料

# 抓取今年及季
year_AD=datetime.datetime.today().strftime("%Y")
season_now=datetime.datetime.today().strftime("%m-%d")

# 第一季 5 月 16 日更新，半年度 8 月 15 日更新，第三季 11 月 15 日更新，年度 4 月 1 日更新。
if(season_now<"05-16"):
  season_now=0
elif(season_now<"08-05"):
  season_now=1
elif(season_now<"11-16"):
  season_now=2
elif(season_now>="11-16"):
  season_now=3
else:
  season_now=0
  
# 轉換成民國年份
year =int(year_AD)-1911;

result = {} 

# 逐年逐季抓取資料
for yr in range(3):
  y=year-yr
  for season in range(4):
    s=season+1
    if(y!=year):
      result['%d-%d'%(y,s)] = fetch_financial_statements(y, s)
    else:
      if(s<season):
        result['%d-%d'%(y,s)] = fetch_financial_statements(y, s)
        

for key in result.keys():
  result[key].index = result[key]['公司代號']

obj={}
for index,df in result.items():
  obj[index]=df['毛利率(%)(營業毛利)/(營業收入)']

df = pd.DataFrame.from_dict(obj,orient='index')
df = df.sort_index()
df

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,6668,6669,6672,6674,8104,8482,1341,6666,6671,3712
106-1,14.16,7.45,22.15,12.63,9.03,14.40,-2.87,26.14,30.99,13.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106-2,18.01,12.07,16.82,12.55,9.14,19.00,-4.07,28.26,31.09,13.17,...,54.06,4.88,26.51,9.77,26.63,NaN,NaN,NaN,NaN,NaN
106-3,18.16,13.14,15.08,12.89,7.91,19.31,-3.58,30.21,32.37,13.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106-4,19.24,15.67,13.47,12.21,3.74,23.36,-2.52,31.29,32.61,13.48,...,53.20,4.95,25.20,10.84,25.12,42.11,21.65,60.36,38.22,NaN
107-1,23.95,20.09,11.63,11.88,2.47,22.71,8.35,28.20,33.75,14.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107-2,27.93,23.92,14.47,11.46,3.70,29.27,8.58,30.19,32.76,14.25,...,41.88,6.15,24.04,12.92,23.94,38.80,NaN,NaN,NaN,NaN
107-3,27.51,24.51,14.68,11.79,2.83,30.55,8.00,31.50,32.56,14.20,...,43.81,6.03,24.47,13.33,22.15,40.13,16.44,65.71,38.08,NaN
107-4,26.96,25.59,13.79,11.94,1.40,32.59,8.62,31.07,32.40,13.93,...,40.41,5.88,24.86,13.63,20.66,40.58,17.26,67.04,37.52,9.79
